# A Two-State HMM: Vowels and Consonants
## Part 1.
1. Code for setting up the states.
2. Initialize A,B,Pi as distributions. 
3. Calculate alpha and beta.
4. Calculate the probability of each word in two ways so they match

## Part 2. Expectation & Part 3. Maximization
### Training objective: minimize total plog (negative base-2 log)
1. Calculate the soft (or expected) count of each letter.
2. Set up tables for soft counts in general, and word-initially
3. Recomputing A, B, and Π.
4. Putting expectation and maximization into a loop sensibly, with a stop condition that makes sense.
5. Show the preference of each letter for one of the states by computing the log ratios of the emission proabilities
6. Determine the values of A for the best analysis (highest probability) of the English data.

## Part 4. Viterbi

In [15]:
import numpy as np
import pandas as pd
from collections import deque
np.random.seed(25020)

In [2]:
def read_corpus(file):
    with open(file, 'rt') as f:
        lines = f.readlines()
    lines = [line.rstrip().lower() + '#' for line in lines]
    return lines

def create_lookup(corpus):
    alphabet = set()
    for word in corpus:
        alphabet.update(list(word))
    idx_to_char_dict = sorted(alphabet)
    char_to_idx_dict = {char: idx for idx, char in enumerate(idx_to_char_dict)}
    return idx_to_char_dict, char_to_idx_dict

# encode and decode sequence
def encode(lookup, seq):
    ret = np.array([lookup[char] for char in seq]) # an idx array
    return ret

In [22]:
class HMM:
    def __init__(self, num_states, num_emissions):
        self.num_states = num_states
        self.num_emissions = num_emissions
        
        self.pi = np.random.random(num_states).astype('float128')
        self.A = np.random.random((num_states, num_states)).astype('float128')
        self.B = np.random.random((num_emissions, num_states)).astype('float128')
        # normalize probabilities
        self.pi /= self.pi.sum() # sum up to 1
        self.A /= self.A.sum(axis=1) # row summing to 1
        self.B /= self.B.sum(axis=0) # col summing to 1
        
        self.alpha = None # forward
        self.beta = None # backward
        self.xi = None # EM
        self.gamma = None # EM
        
        self.plog = 0
        
    def train(self, seqs):
        self.plog = 0
        self.pi_update = np.zeros(self.num_states)
        self.A_update = np.zeros((self.num_states, self.num_states)) 
        self.B_update = np.zeros((self.num_emissions, self.num_states))
        num_seq = 0
        for seq in seqs:
            self.expectation(seq)
            self.maximization(seq)
            num_seq += 1
        # acutally apply the maximization updates
        self.pi = self.pi_update / num_seq
        self.A = self.A_update / num_seq
        self.B = self.B_update / num_seq

        return self.plog
        
    def forward(self, seq):
        T = seq.shape[0]
        self.alpha = np.empty((T, self.num_states), dtype='float128') # T * num_states
        self.alpha[0] = self.pi * self.B[seq[0]]
        for t in range(1, T):
            self.alpha[t] = self.alpha[t - 1] @ self.A * self.B[seq[t]]
    
    def backward(self, seq):
        num_states = self.A.shape[0]
        T = seq.shape[0]
        self.beta = np.empty((T, self.num_states), dtype='float128')
        self.beta[T - 1] = 1
        for t in range(T - 2, -1, -1):
            self.beta[t] = self.A * self.B[seq[t + 1]] @ self.beta[t + 1]
            
    def alpha_prob(self):
        return sum(self.alpha[-1])
    
    def beta_prob(self, seq):
        return sum(self.pi * self.B[seq[0]] * self.beta[0])
    
    def expectation(self, seq):
        self.forward(seq)
        self.backward(seq)
        self.plog += -np.log2(self.alpha_prob())
        
        alpha_p = self.alpha_prob()
        T = seq.shape[0]
        self.xi = np.empty((T - 1, self.num_states, self.num_states))
        for t in range(T - 1):
            self.xi[t] = np.outer(self.alpha[t], self.beta[t + 1]) * \
            self.A * self.B[seq[t + 1]]
        self.xi /= alpha_p
        
        self.gamma = self.alpha * self.beta / alpha_p
    
    def maximization(self, seq):
        # record updates based on expectations
        # update pi
        self.pi_update += self.gamma[0] # soft counts of each state at time 1
        # update A
        numerator = self.xi.sum(axis=0) # soft counts of transitions from i to j
        denom = self.gamma[:-1].sum(axis=0) # soft counts of transitions out of i
        self.A_update += numerator / denom[:, None]
        # update B
        temp = np.empty((self.num_emissions, self.num_states), dtype='float128')
        for k in range(self.num_emissions):
            temp[k] = self.gamma[seq == k].sum(axis=0)
        self.B_update += temp / denom
        
    def viterbi(self, seq):
        T = seq.shape[0]
        delta = np.empty((T, self.num_states))
        psi = np.empty((T, self.num_states), dtype='int')
        psi[0] = np.arange(self.num_states) # best state to come from initially, aka. pi
        delta[0] = self.pi * self.B[seq[0]]
        for t in range(1, T):
            temp = delta[t - 1][:, None] * self.A
            psi[t] = temp.argmax(axis=0)
            delta[t] = temp.max(axis=0) * self.B[seq[t]]
        
        # back trace
        best = np.argmax(delta[-1])
        path = deque([best])
        for t in range(T - 2, -1, -1):
            best = psi[t, best]
            path.appendleft(best)
        return path
        
    def check_probabilities(self):
        assert np.isclose(self.pi.sum(), 1)
        assert np.allclose(self.A.sum(axis=1), 1)
        assert np.allclose(self.B.sum(axis=0), 1)

# Training

In [23]:
corpus = read_corpus('english1000.txt')
idx_to_char_dict, char_to_idx_dict = create_lookup(corpus)
num_states = 2
seqs = [encode(char_to_idx_dict, word) for word in corpus]
hmm = HMM(num_states, len(idx_to_char_dict))

In [52]:
for i in range(51):
    plog = hmm.train(seqs)
    if i % 10 == 0:
        print(i, plog)

0 21792.700765342927905
10 21696.766399786400676
20 21594.704560289753843
30 21486.081945886238382
40 21370.76880603925473
50 21248.797549298314069


In [25]:
# analyze log ratios of emissions from the two states
np.set_printoptions(precision=3, suppress=True)
error = 1e-20 # prevent division by 0
logs = np.log(hmm.B[:, 0] / hmm.B[:, 1] + error)
letters = [idx_to_char_dict[idx] for idx in range(logs.shape[0])]
df = pd.DataFrame.from_records({'Letter': letters, 'Log Ratio': logs})

In [53]:
df[df['Log Ratio'] > 0].sort_values(by='Log Ratio', ascending=False)

,Letter,Log Ratio
19,q,165.464407
12,j,3.479090
4,b,2.633730
25,w,1.624656
8,f,1.609132
21,s,1.198874
5,c,1.057449
18,p,0.886704
15,m,0.861642
9,g,0.624906


In [54]:
df[df['Log Ratio'] <= 0].sort_values(by='Log Ratio')

,Letter,Log Ratio
0,#,-46.051702
26,x,-46.051702
28,z,-46.051702
1,',-46.051702
2,.,-46.051702
7,e,-1.485382
23,u,-1.380228
17,o,-1.345887
24,v,-1.315089
16,n,-1.091230


In [115]:
# comparing the performance of 25 different intializations
# for each of the two states, self-transitioning prob: 0.1, 0.3, 0.5, 0.7, 0.9
train_iter = 5
np.set_printoptions(precision=3, suppress=True)
for i in np.arange(0.1, 1, 0.2): # state 0 self-transition
    for j in np.arange(0.1, 1, 0.2): # state 1 self-transition
        hmm = HMM(num_states, len(idx_to_char_dict))
        hmm.A = np.array([[i, 1 - i], [j, 1 - j]], dtype='float128')
        for k in range(train_iter):
            plog = hmm.train(seqs)
        print('State 0 self-trans: {}, State 1 self-trans: {}\nPlog:{}\n\n'.
              format(i, j, plog))

State 0 self-trans: 0.1, State 1 self-trans: 0.1
Plog:22173.96922966324


State 0 self-trans: 0.1, State 1 self-trans: 0.30000000000000004
Plog:22465.570753671964


State 0 self-trans: 0.1, State 1 self-trans: 0.5000000000000001
Plog:22527.511442770265


State 0 self-trans: 0.1, State 1 self-trans: 0.7000000000000001
Plog:22761.293473739057


State 0 self-trans: 0.1, State 1 self-trans: 0.9000000000000001
Plog:22573.176903708074


State 0 self-trans: 0.30000000000000004, State 1 self-trans: 0.1
Plog:22087.248760886574


State 0 self-trans: 0.30000000000000004, State 1 self-trans: 0.30000000000000004
Plog:22671.671609692523


State 0 self-trans: 0.30000000000000004, State 1 self-trans: 0.5000000000000001
Plog:22921.4403769643


State 0 self-trans: 0.30000000000000004, State 1 self-trans: 0.7000000000000001
Plog:22571.205947703893


State 0 self-trans: 0.30000000000000004, State 1 self-trans: 0.9000000000000001
Plog:22035.788479415805


State 0 self-trans: 0.5000000000000001, State 1 sel

In [64]:
test = encode(char_to_idx_dict, 'qqqqqq#')
hmm.viterbi(test)

deque([0, 0, 0, 0, 0, 0, 1])

In [67]:
test = encode(char_to_idx_dict, 'qxxxxxxqqqq#')
hmm.viterbi(test)

deque([0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1])